In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filepath = "/content/drive/MyDrive/iot_23_datasets_small/opt/Malware-Project/BigDataset/iot23_combined.csv"
df = pd.read_csv(filepath)

In [ ]:
df


,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1,1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
2,2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
3,3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,...,False,False,False,True,False,False,False,False,False,False
4,4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444669,99994,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1444670,99995,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1444671,99996,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1444672,99997,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False


In [ ]:
del df['Unnamed: 0']

In [ ]:
df['label'].value_counts()

label
PartOfAHorizontalPortScan     825939
Okiru                         262690
Benign                        197809
DDoS                          138777
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']].values

In [ ]:
X.shape

(1444674, 24)

In [ ]:
Y = pd.get_dummies(df['label']).values

In [ ]:
Y.shape

(1444674, 12)

In [ ]:
X


array([[2.998796, 0, 0, ..., False, False, False],
       [0.0, 0, 0, ..., False, False, False],
       [0.0, 0, 0, ..., False, False, False],
       ...,
       [0.0, 0, 0, ..., False, False, False],
       [0.0, 0, 0, ..., False, False, False],
       [0.0, 0, 0, ..., False, False, False]], dtype=object)

In [ ]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,False,...,False,False,False,True,False,False,False,False,False,False
1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,False,...,False,False,False,True,False,False,False,False,False,False
2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,False,...,False,False,False,True,False,False,False,False,False,False
3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,False,...,False,False,False,True,False,False,False,False,False,False
4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444669,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,False,...,False,False,False,True,False,False,False,False,False,False
1444670,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,False,...,False,False,False,True,False,False,False,False,False,False
1444671,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,False,...,False,False,False,True,False,False,False,False,False,False
1444672,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,False,...,False,False,False,True,False,False,False,False,False,False


In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X)

MinMaxScaler()

In [ ]:
normalized_x = scaler.transform(X)


In [ ]:
normalized_x

array([[8.16450401e-05, 5.73121586e-10, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 5.73121586e-10, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 5.73121586e-10, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.04174057e-05, 5.73121586e-10, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 5.73121586e-10, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 5.73121586e-10, 8.57558209e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
normalized_x.shape

(1444674, 24)

In [ ]:
scaler.fit(Y)

MinMaxScaler()

In [ ]:
normalized_y = scaler.transform(Y)

In [ ]:
normalized_y

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [ ]:
X_train.shape

(1155739, 24)

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(2000, activation='relu',input_dim=24))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              50000     
                                                                 
 dense_1 (Dense)             (None, 1500)              3001500   
                                                                 
 dropout (Dropout)           (None, 1500)              0         
                                                                 
 dense_2 (Dense)             (None, 800)               1200800   
                                                                 
 dropout_1 (Dropout)         (None, 800)               0         
                                                                 
 dense_3 (Dense)             (None, 400)               320400    
                                                                 
 dropout_2 (Dropout)         (None, 400)               0

In [ ]:
import time

In [ ]:
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, epochs = 10, batch_size=256, validation_data=(X_test,Y_test),verbose=1)

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...

Epoch 1/10
4515/4515 [==============================] - 1038s 230ms/step - loss: 0.8658 - accuracy: 0.6933 - val_loss: 0.8609 - val_accuracy: 0.6934
Epoch 2/10
4515/4515 [==============================] - 1063s 236ms/step - loss: 0.8600 - accuracy: 0.6938 - val_loss: 0.8622 - val_accuracy: 0.6934
Epoch 3/10
4515/4515 [==============================] - 1017s 225ms/step - loss: 0.8593 - accuracy: 0.6939 - val_loss: 0.8609 - val_accuracy: 0.6934
Epoch 4/10
4515/4515 [==============================] - 1031s 228ms/step - loss: 0.8590 - accuracy: 0.6939 - val_loss: 0.8608 - val_accuracy: 0.6934
Epoch 5/10
4515/4515 [==============================] - 1000s 221ms/step - loss: 0.8588 - accuracy: 0.6939 - val_loss: 0.8605 - val_accuracy: 0.6934
Epoch 6/10
4515/4515 [==============================] - 1058s 234ms/step - loss: 0.8586 - accuracy: 0.6939 - val_loss: 0.8612 - val_accuracy: 0.6934
Epoch 7/10
4515/4515 [==============================] - 1030s 228ms/step - loss: 0.8585 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.3)

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(10000, activation='relu',input_dim=24))
model.add(Dense(5000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10000)             250000    
                                                                 
 dense_7 (Dense)             (None, 5000)              50005000  
                                                                 
 dropout_4 (Dropout)         (None, 5000)              0         
                                                                 
 dense_8 (Dense)             (None, 2500)              12502500  
                                                                 
 dropout_5 (Dropout)         (None, 2500)              0         
                                                                 
 dense_9 (Dense)             (None, 1000)              2501000   
                                                                 
 dropout_6 (Dropout)         (None, 1000)             

In [ ]:
import time
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, epochs = 1, batch_size=256, validation_data=(X_test,Y_test),verbose=1)

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...

1054/3951 [=======>......................] - ETA: 4:07:15 - loss: 0.8957 - accuracy: 0.6898